# Full Stats, Haccytrees, Abundances
This notebook will be like a combination of Haccytrees.ipynb and LJ-Full_Fossil_Groups.ipynb. After this one is finished, those notebooks may be cleaned up or deleted

## Set Up

In [2]:
import haccytrees.mergertrees
import h5py
import numpy as np
import numba
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import pandas as pd
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from itertools import groupby
from matplotlib.ticker import ScalarFormatter
%load_ext line_profiler
%reload_ext autoreload
%autoreload 1
%aimport help_func_haccytrees

plt.rcParams.update({
    "text.usetex": True,
    'font.size': 13,
    "figure.figsize": (5.25, 3.5),#(6.25, 4.25), #(6.25, 3.75)
    "patch.linewidth": 1
})

pyfftw not available, using numpy fft


In [ ]:
%%time
forest, progenitor_array = haccytrees.mergertrees.read_forest(
    '/data/a/cpac/mbuehlmann/LastJourney/forest/target_forest_aurora.hdf5',
    'LastJourney', nchunks=1, chunknum=0, mass_threshold = 2.7*10**11,
)

In [ ]:
%%time
fg_forest_lm5, fg_progenitor_array_lm5 = haccytrees.mergertrees.read_forest(
    "/data/a/cpac/mbuehlmann/LastJourney/forest/fg_forest.hdf5",
    'LastJourney',
    mass_threshold=5e11
)

## Create masks (for narrow mass bins)

In [ ]:
halo_masks = help_func_haccytrees.make_masks(forest)
fg_masks_lm5 = help_func_haccytrees.make_masks(fg_forest_lm5)

## Substructure stats

In [30]:
forest, progenitor_array = haccytrees.mergertrees.read_forest(
    '/data/a/cpac/mbuehlmann/LastJourney/forest/target_forest_aurora.hdf5',
    'LastJourney', nchunks=1, chunknum=0, mass_threshold = 2.7*10**11,
    include_fields = ["tree_node_mass", "snapnum", "fof_halo_tag", "sod_halo_cdelta", "fof_halo_center_x", "fof_halo_center_y", "fof_halo_center_z"]
)

In [31]:
# Add columns "fsubmax" and "fsubtot" to my forests
subhalo_data = haccytrees.mergertrees.create_submass_data(
    forest,
    'LastJourney',
    mass_threshold=1e10,
    compute_fsub_stats=True)

fg_subhalo_data = haccytrees.mergertrees.create_submass_data(
    fg_forest_lm5,
    'LastJourney',
    mass_threshold=1e10,
    compute_fsub_stats=True)

In [33]:
full_halo_masks = help_func_haccytrees.make_masks(forest) # old ones were built on data

In [35]:
# Calculate average fsubstats for each population in each mass bin
for i, (these_masks, this_forest) in enumerate(zip([full_halo_masks, fg_masks_lm5], [forest, fg_forest_lm5])): # loop over fossil status
    for j, this_mask in enumerate(these_masks): # loop over mass bins
        print(i, j)
        fsubmax_lst = this_forest['fsubmax'][this_mask]
        fsubtot_lst = this_forest['fsubtot'][this_mask]
        avg_fsubmax = np.average(fsubmax_lst)
        avg_fsubtot = np.average(fsubtot_lst)
        std_fsubmax = np.std(fsubmax_lst)
        std_fsubtot = np.std(fsubtot_lst)
        se_fsubmax = std_fsubmax/np.sqrt(len(fsubmax_lst))
        se_fsubtot = std_fsubtot/np.sqrt(len(fsubtot_lst))
        print(len(fsubmax_lst))
        print("fsubmax: ", avg_fsubmax, " \pm ", se_fsubmax)
        print("fsubtot: ", avg_fsubtot, " \pm ", se_fsubtot)

0 0
269358
fsubmax:  0.021441238  \pm  6.266740489238504e-05
fsubtot:  0.05058351  \pm  0.00013128643561581535
0 1
36181
fsubmax:  0.034970697  \pm  0.00019054605435978237
fsubtot:  0.087785505  \pm  0.0004061289263832541
0 2
2454
fsubmax:  0.049261373  \pm  0.0009176684228133129
fsubtot:  0.12952824  \pm  0.0017769909000801445
1 0
269358
fsubmax:  0.0030381705  \pm  5.243849033110079e-06
fsubtot:  0.00496545  \pm  1.0086163010781129e-05
1 1
36181
fsubmax:  0.003285462  \pm  2.78423814909428e-05
fsubtot:  0.008101042  \pm  6.329417015924221e-05
1 2
2454
fsubmax:  0.009164177  \pm  0.00022960044541477697
fsubtot:  0.021776568  \pm  0.0004846684913295319


## Side Quest: Halo radius

In [26]:
bin_names = ["13.0 - 13.05", "13.3 - 13.35", "13.6 - 13.65"]
for i, (this_halo_mask, this_fg_mask) in enumerate(zip(halo_masks, fg_masks_lm5)):
    print(bin_names[i])
    halo_radii = forest['sod_halo_radius'][this_halo_mask]
    fg_radii = fg_forest_lm5['sod_halo_radius'][this_fg_mask]
    avg_halo_r = np.average(halo_radii)
    avg_fg_r = np.average(fg_radii)
    se_halo_r = np.std(halo_radii)/np.sqrt(len(halo_radii))
    se_fg_r = np.std(fg_radii)/np.sqrt(len(fg_radii))
    print("Average halo: ", avg_halo_r, "+/-", se_halo_r)
    print("Average FG:   ", avg_fg_r, "+/-", se_fg_r)

13.0 - 13.05
Average halo:  0.3342302 +/- 4.0555591500348715e-05
Average FG:    0.3470978 +/- 1.2823666379149224e-05
13.3 - 13.35
Average halo:  0.4194501 +/- 0.0001428139618961365
Average FG:    0.43954965 +/- 3.833760134243303e-05
13.6 - 13.65
Average halo:  0.5250816 +/- 0.0007098247347622455
Average FG:    0.5564692 +/- 0.00017027119214394133
